# Calgary Property Assessment Data - Setup
### DO NOT MODIFY - RUN "AS IS"

This notebook loads Calgary property assessment data from CSV files into Unity Catalog.

**Target Table**: `calgary_real_estate.property_assessments.2025_data`

In [ ]:
%pip install pandas
dbutils.library.restartPython()

In [ ]:
import pandas as pd

In [ ]:
# Unity Catalog configuration
catalog = "calgary_real_estate"
schema = "property_assessments"
table = "2025_data"

In [ ]:
# Create catalog and schema
spark.sql(f"""CREATE CATALOG IF NOT EXISTS {catalog}""")
spark.sql(f"""CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}""")

print(f"✓ Catalog '{catalog}' created")
print(f"✓ Schema '{catalog}.{schema}' created")

In [ ]:
# Load Calgary property data from split CSV files
csv_files = [
    "../data/calgary_property_data_full_part1.csv",
    "../data/calgary_property_data_full_part2.csv",
    "../data/calgary_property_data_full_part3.csv",
    "../data/calgary_property_data_full_part4.csv"
]

print("Reading Calgary property data CSV files...\n")

# Read all CSV parts and combine them
dfs = []
for i, csv_file in enumerate(csv_files, 1):
    print(f"Reading part {i}/{len(csv_files)}: {csv_file}")
    df = pd.read_csv(csv_file, low_memory=False)
    dfs.append(df)
    print(f"  → Loaded {len(df):,} rows")

# Combine all DataFrames
print("\nCombining all CSV parts...")
combined_df = pd.concat(dfs, ignore_index=True, sort=False)
print(f"✓ Total records: {len(combined_df):,}\n")

# Convert to Spark DataFrame
print("Converting to Spark DataFrame...")
spark_df = spark.createDataFrame(combined_df)

# Save as Delta table
table_name = f"{catalog}.{schema}.{table}"
print(f"Saving to Unity Catalog table: {table_name}...")
spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(table_name)

print(f"\n✓ Successfully created table: {table_name}")
print(f"✓ Total records in table: {spark.table(table_name).count():,}")

In [ ]:
# Display sample data
print("Sample data from table:")
display(spark.table(f"{catalog}.{schema}.{table}").limit(10))

In [ ]:
# Data quality summary
print("Data Quality Summary:")
df = spark.table(f"{catalog}.{schema}.{table}")
print(f"Total rows: {df.count():,}")
print(f"Total columns: {len(df.columns)}")
print(f"\nColumn names:")
for col in df.columns:
    print(f"  - {col}")